In [1]:
import pandas as pd
import numpy as np
import yaml
from os import getcwd,chdir

In [4]:
with open('../excel_files/input/data.yml') as file:
    yml_data = yaml.load(file, Loader=yaml.FullLoader)
file.close()

In [5]:
yml_data

{'directory': 'C:\\\\Users\\\\Sahil\\\\Documents\\\\spacemonk\\\\excel_parser\\\\excel_files\\\\input',
 'filename': '../excel_files/modified_original/Summit 52 week PPM Schedule-2019.xlsx',
 'upper matrix header': 'activity description',
 'coordinate term': 'frequency'}

In [7]:
chdir(yml_data['directory'])
getcwd()

'C:\\Users\\Sahil\\Documents\\spacemonk\\excel_parser\\excel_files\\input'

In [11]:
import os

In [21]:
for files in os.walk('C:\\Users\\Sahil\\Documents\\spacemonk\\excel_parser\\excel_files\\input'):
    for each_file in files[2]:
        print(each_file)
        if ".xlsx" in each_file:
            print("files found")

data.yml
Summit 52 week PPM Schedule-2019.xlsx
files found


In [ ]:
yml_data

In [ ]:
file = pd.ExcelFile(yml_data["filename"])

In [ ]:
df = pd.read_excel(file,sheet_name=file.sheet_names[0],header = None)

In [ ]:
# drops the complete null rows and columns in a df
def df_dropna(df):
    df.dropna(axis=0,how="all",inplace=True)
    df.dropna(axis=1,how="all",inplace=True)


In [ ]:
df_dropna(df)

In [ ]:
# Mapfunction to convert all the string in df to lowercase and keeping the other types same
def df_to_lower(df):
    return df.applymap(lambda s:s.lower() if type(s) == str else s)

In [ ]:
df =df_to_lower(df)

In [ ]:
# setting the index of df back from 0
def reset_index(df):
    df.reset_index(inplace=True)
    df.drop(df.columns[0], axis=1,inplace=True)

In [ ]:
#setting the header, by the first row
def set_column(df):
    df.columns = df.iloc[0]
    df.drop(df.index[0],inplace=True)

In [ ]:
# function to slice the header having the logo and other unuseful information
def upper_matrix_slice(df,header):
    x =df.iloc[:,0].str.find(header.lower()) # find() returns complete index of series, searches in first column of df
    x=int(x.index[x==0][0]) # find the true index of value and returns int 
    df =df.iloc[x:,:] # slicing the df before the header row
    return df

In [ ]:
# function to get first occurance coordinates (row,column) of a string in a dataframe
def coordinates(df,term):
    for i in range(len(df.columns)):
        x=df[df.columns[i]].str.find(term)
        if(len(x.index[x==0])==0):
            pass
        else:
            x=int(x.index[x==0][0])
            return (x,i)  

In [ ]:
def week_list(df):
  # finds the count of non nan
  for i in range(1,df.shape[1]+1):
        df.loc[df[i]==df[i],i]=i
  week_num = [[ j for j in i if j ==j] for i in np.array(df) ]
  return week_num

In [ ]:
df =upper_matrix_slice(df,yml_data['upper matrix header'])

In [ ]:
reset_index(df)

In [ ]:
freq_c =coordinates(df,yml_data['coordinate term'])

In [ ]:
df_freq = df.iloc[freq_c[0]:,freq_c[1]+1:]

In [ ]:
df_act = df.iloc[0:,0:freq_c[1]+1]

In [ ]:
set_column(df_act)

In [ ]:
set_column(df_freq)

In [ ]:
df_act["week number"] = week_list(df_freq)

In [ ]:
df_act.loc[df_act.frequency!=df_act.frequency ,'week number'] = "---"
df_act.loc[df_act.frequency=='daily' ,'week number'] = "d"
df_act.loc[df_act.frequency=='weekly' ,'week number'] = "w"

In [ ]:
df_act.to_excel("../excel_files/workorder/"+file.sheet_names[0]+".xlsx")